<a href="https://colab.research.google.com/github/Waleed850/Coding-Code/blob/main/Crop_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data **Preprocessing**

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import numpy as np
from osgeo import gdal, gdal_array
import pandas as pd
import operator
import gc
import os

In [2]:
# Open the region of interest (ROI) file
roi_ds = gdal.Open('/content/drive/MyDrive/GDToT/TCI.jp2', gdal.GA_ReadOnly)
roi = roi_ds.GetRasterBand(1).ReadAsArray()

In [3]:
# Find unique classes and their pixel counts
classes, counts = np.unique(roi, return_counts=True)
class_counts = dict(zip(classes, counts))
sorted_classes = sorted(class_counts.items(), key=lambda item: item[1], reverse=True)
print("Top 6 classes and pixel counts \n", sorted_classes[:6])


Top 6 classes and pixel counts 
 [(0, 87646334), (154, 514698), (152, 514025), (153, 513500), (150, 512870), (155, 512863)]


In [4]:
# Select top 5 classes excluding class label 255
top_classes = [155, 150, 153, 152, 154]
# Count non-zero entries (training samples)
n_samples = np.count_nonzero(roi)
print(f'We have {n_samples} samples')

We have 32914066 samples


In [6]:
# Get classification labels
labels = np.unique(roi[roi > 0])

In [7]:
# Define image file paths
images = [
    '/content/drive/MyDrive/GDToT/B01.jp2',
    '/content/drive/MyDrive/GDToT/B02.jp2',
    '/content/drive/MyDrive/GDToT/B03.jp2',
    '/content/drive/MyDrive/GDToT/B04.jp2',
    '/content/drive/MyDrive/GDToT/B05.jp2']

In [8]:
print("Reading class 163")
final = pd.DataFrame()

for c in top_classes:

    temp = pd.DataFrame()

    print(c)
    for img in images:

        print(img)

        train_ds = gdal.Open(img, gdal.GA_ReadOnly)

        print(train_ds.RasterXSize,train_ds.RasterYSize)

        img_b1 = np.zeros((train_ds.RasterYSize, train_ds.RasterXSize, train_ds.RasterCount),
                       gdal_array.GDALTypeCodeToNumericTypeCode(train_ds.GetRasterBand(1).DataType))

        for b in range(img_b1.shape[2]):
            img_b1[:, :, b] = train_ds.GetRasterBand(b + 1).ReadAsArray()

        print(img_b1.shape)
        roi_resized = np.resize(roi, (img_b1.shape[0],))  # Resize along the first dimension only
        Xt = img_b1[roi_resized == c, :]

        # Reshape Xt to 2 dimensions
        Xt_2d = Xt.reshape((Xt.shape[0], -1))  # Flatten the last two dimensions

        # Create DataFrame
        Xt1 = pd.DataFrame(Xt_2d)

        # Sample with replacement
        Xt2 = Xt1.sample(n=min(100000, len(Xt1)), replace=True)
        Xt2.reset_index(drop=True,inplace=True)

        temp = pd.concat([Xt2,temp],axis=1)

        temp["class"] = c
        #temp.reset_index(drop=True,inplace=True)

    final = pd.concat([temp,final],axis=0)
    final.reset_index(drop=True,inplace=True)

    gc.collect()

Reading class 163
155
/content/drive/MyDrive/GDToT/B01.jp2
1830 1830
(1830, 1830, 1)
/content/drive/MyDrive/GDToT/B02.jp2
10980 10980
(10980, 10980, 1)
/content/drive/MyDrive/GDToT/B03.jp2
10980 10980
(10980, 10980, 1)
/content/drive/MyDrive/GDToT/B04.jp2
10980 10980
(10980, 10980, 1)
/content/drive/MyDrive/GDToT/B05.jp2
5490 5490
(5490, 5490, 1)
150
/content/drive/MyDrive/GDToT/B01.jp2
1830 1830
(1830, 1830, 1)
/content/drive/MyDrive/GDToT/B02.jp2
10980 10980
(10980, 10980, 1)
/content/drive/MyDrive/GDToT/B03.jp2
10980 10980
(10980, 10980, 1)
/content/drive/MyDrive/GDToT/B04.jp2
10980 10980
(10980, 10980, 1)
/content/drive/MyDrive/GDToT/B05.jp2
5490 5490
(5490, 5490, 1)
153
/content/drive/MyDrive/GDToT/B01.jp2
1830 1830
(1830, 1830, 1)
/content/drive/MyDrive/GDToT/B02.jp2
10980 10980
(10980, 10980, 1)
/content/drive/MyDrive/GDToT/B03.jp2
10980 10980
(10980, 10980, 1)
/content/drive/MyDrive/GDToT/B04.jp2
10980 10980
(10980, 10980, 1)
/content/drive/MyDrive/GDToT/B05.jp2
5490 5490
(5490

In [30]:
# Save the final DataFrame to CSV
final.to_csv("Dataset123.csv", index=False)

In [31]:
final

,0,1,2,3,4,5,6,7,8,9,...,1821,1822,1823,1824,1825,1826,1827,1828,1829,class
0,2589,2500,2113,1747,2158,2189,1925,1707,1618,1712,...,1226.0,1201.0,1189.0,1228.0,1295.0,1302.0,1276.0,1241.0,1173.0,154
1,1660,1573,1446,1427,1842,1827,1802,1803,1670,1600,...,1147.0,1150.0,1158.0,1178.0,1169.0,1172.0,1241.0,1270.0,1274.0,154
2,2366,2374,2416,2397,2379,2393,2313,2167,2257,2364,...,1100.0,1104.0,1105.0,1091.0,1093.0,1096.0,1092.0,1092.0,1094.0,154
3,2598,2577,2563,2514,2484,2546,2538,2473,2494,2441,...,1196.0,1222.0,1244.0,1188.0,1194.0,1219.0,1222.0,1250.0,1282.0,154
4,955,1074,1049,1041,1007,1064,987,969,1015,864,...,1095.0,1096.0,1093.0,1083.0,1092.0,1099.0,1096.0,1095.0,1100.0,154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,964,1095,1167,1136,1154,1705,1871,1942,1956,1966,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,155
172,2379,2341,2327,2328,2310,2166,2239,2272,2319,2246,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,155
173,1946,1965,2007,2088,2123,2102,2131,2112,2200,2312,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,155
174,2026,2142,2168,2172,2293,2372,2373,2300,2333,2455,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,155


Crop Classification

In [44]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras import regularizers

In [45]:
# Load the dataset
final = pd.read_csv("/content/Dataset123.csv")
final.columns = ['col_' + str(i) for i in range(40261)]

In [46]:
# Separate features and labels
data = final.iloc[:, :-1]
labels = final['col_40260']

In [47]:
# Encode class values as integers
encoder = LabelEncoder()
encoded_Y = encoder.fit_transform(labels)
dummy_y = to_categorical(encoded_Y, num_classes=5)

In [48]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(data, dummy_y, test_size=0.33, random_state=42, shuffle=True)

In [49]:
# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [50]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras import regularizers

In [51]:
# Build the model
model = Sequential()
model.add(Dense(200, input_shape=(X_train.shape[1],), activation='relu',
                kernel_regularizer=regularizers.l2(1e-5),
                kernel_initializer='glorot_normal', bias_initializer='zeros'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

In [52]:

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

In [53]:
final.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,col_40251,col_40252,col_40253,col_40254,col_40255,col_40256,col_40257,col_40258,col_40259,col_40260
0,2589,2500,2113,1747,2158,2189,1925,1707,1618,1712,...,1226.0,1201.0,1189.0,1228.0,1295.0,1302.0,1276.0,1241.0,1173.0,154
1,1660,1573,1446,1427,1842,1827,1802,1803,1670,1600,...,1147.0,1150.0,1158.0,1178.0,1169.0,1172.0,1241.0,1270.0,1274.0,154
2,2366,2374,2416,2397,2379,2393,2313,2167,2257,2364,...,1100.0,1104.0,1105.0,1091.0,1093.0,1096.0,1092.0,1092.0,1094.0,154
3,2598,2577,2563,2514,2484,2546,2538,2473,2494,2441,...,1196.0,1222.0,1244.0,1188.0,1194.0,1219.0,1222.0,1250.0,1282.0,154
4,955,1074,1049,1041,1007,1064,987,969,1015,864,...,1095.0,1096.0,1093.0,1083.0,1092.0,1099.0,1096.0,1095.0,1100.0,154


In [55]:
# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=100, shuffle=True)


Epoch 1/100
2/2 [==============================] - 0s 104ms/step - loss: nan - accuracy: 0.1368
Epoch 2/100
2/2 [==============================] - 0s 101ms/step - loss: nan - accuracy: 0.1368
Epoch 3/100
2/2 [==============================] - 0s 99ms/step - loss: nan - accuracy: 0.1368
Epoch 4/100
2/2 [==============================] - 0s 103ms/step - loss: nan - accuracy: 0.1368
Epoch 5/100
2/2 [==============================] - 0s 99ms/step - loss: nan - accuracy: 0.1368
Epoch 6/100
2/2 [==============================] - 0s 96ms/step - loss: nan - accuracy: 0.1368
Epoch 7/100
2/2 [==============================] - 0s 115ms/step - loss: nan - accuracy: 0.1368
Epoch 8/100
2/2 [==============================] - 0s 98ms/step - loss: nan - accuracy: 0.1368
Epoch 9/100
2/2 [==============================] - 0s 100ms/step - loss: nan - accuracy: 0.1368
Epoch 10/100
2/2 [==============================] - 0s 105ms/step - loss: nan - accuracy: 0.1368
Epoch 11/100
2/2 [=========================

In [56]:
# Evaluate the model
score = model.evaluate(X_test, y_test, batch_size=32)
print("\n%s: %.2f%%" % (model.metrics_names[1], score[1] * 100))

2/2 [==============================] - 0s 19ms/step - loss: nan - accuracy: 0.1864

accuracy: 18.64%


NVDI BASED

In [ ]:
from osgeo import gdal, gdal_array
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from keras.optimizers import Adadelta, Adam
from keras import regularizers, initializers
import tensorflow as tf

# Tell GDAL to throw Python exceptions, and register all drivers
gdal.UseExceptions()
gdal.AllRegister()

# Load image and ROI datasets
img_ds = gdal.Open('/content/drive/MyDrive/GDToT/T34JEP_20170101T082332_B01.jp2', gdal.GA_ReadOnly)
roi_ds = gdal.Open('/content/drive/MyDrive/GDToT/T34JEP_20170101T082332_TCI.jp2', gdal.GA_ReadOnly)

# Read image data into a NumPy array
img = np.zeros((img_ds.RasterYSize, img_ds.RasterXSize, img_ds.RasterCount),
               gdal_array.GDALTypeCodeToNumericTypeCode(img_ds.GetRasterBand(1).DataType))
for b in range(img.shape[2]):
    img[:, :, b] = img_ds.GetRasterBand(b + 1).ReadAsArray()

# Read ROI data into a NumPy array
roi = roi_ds.GetRasterBand(1).ReadAsArray().astype(np.uint8)

# Display images
plt.subplot(121)
plt.title('Satellite Image')
plt.subplot(122)
plt.imshow(roi)
plt.title('CDL Image')
plt.show()

# Print image and ROI shapes
print(img.shape, roi.shape)

# Display class information
classes = np.unique(roi)
for c in classes:
    n_pixels = (roi == c).sum()  # Count the number of pixels for the current class
    print(f'Class {c} contains {n_pixels} pixels')

# Select top classes
top_classes = [69, 75, 36, 121, 225]

# Read preprocessed final dataset
final = pd.read_csv("/content/Dataset123.csv")
final.columns = ['col_' + str(i) for i in range(6)]

# Separate features and labels
data = final.iloc[:, :-1]
labels = final['col_5']


# Encode class values as integers
encoder = LabelEncoder()
encoded_Y = encoder.fit_transform(labels)
dummy_y = to_categorical(encoded_Y, num_classes=5)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(data, dummy_y, test_size=0.33, random_state=42, shuffle=True)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Calculate NDVI
mul = [i * 5 for i in list(range(11))]
ndvi_df = pd.DataFrame(pd.Series(final.iloc[:, -1]))
for i, m in enumerate(mul[:-1]):
    temp = final.iloc[:, 0 + m:5 + m]
    ndvi = (temp.loc[:, temp.columns[0]] - temp.loc[:, temp.columns[5]]) / (temp.loc[:, temp.columns[5]] + temp.loc[:, temp.columns[0]])
    ndvi_df[f't_{i}'] = ndvi

# Add average NDVI
ndvi_df['avg_ndvi'] = ndvi_df.mean(axis=1)

# Print pivot table for average NDVI per class
pivot_table = pd.pivot_table(ndvi_df, index=["col_5"], values=['avg_ndvi'], aggfunc=np.mean)
print(pivot_table)

# Check NDVI dataframe
print(ndvi_df.head(20))

# Separate NDVI features and labels
X_ndvi = ndvi_df.iloc[:, :-1]
y_ndvi = encoder.transform(ndvi_df['col_5'])

# Split the data for NDVI-based model
X_train_ndvi, X_test_ndvi, y_train_ndvi, y_test_ndvi = train_test_split(X_ndvi, y_ndvi, test_size=0.33, random_state=42, shuffle=True)

# Standardize the NDVI features
X_train_ndvi = scaler.fit_transform(X_train_ndvi)
X_test_ndvi = scaler.transform(X_test_ndvi)
